In [34]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import random
from uuid import uuid4
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import getopt
import sys
import os
import math
import time
import argparse
from visdom import Visdom

from tqdm import tqdm

import pandas as pd
import plotly.graph_objects as go

sys.path.insert(0, os.path.join('..', '..'))

import torch as T
from torch.autograd import Variable as var
import torch.nn.functional as F
import torch.optim as optim

from torch.nn.utils import clip_grad_norm_

from dnc.dnc import DNC
from dnc.sdnc import SDNC
from dnc.sam import SAM
from dnc.util import *

from dnc.lib import *

import copy
import pickle

In [35]:
st = InputStorage()

viz = Visdom()
# assert viz.check_connection()



def llprint(message):
  sys.stdout.write(message)
  sys.stdout.flush()

def debprint(*args):
  DEBUG = False
  if DEBUG:
    print(*args)

def genSeq(sizeTpl, min=None, max=None, default=True):
  if default:
    return np.random.binomial(1, 0.5, sizeTpl)
  assert min is not None and max is not None
  return np.random.randint(min, max, sizeTpl)
  


def generate_data(batch_size, length, maxnumberofcopies=3, currentmaxnocopies=3, testoccurance=True):
  length=length+1
  size=1 # ignore size

  minseq = 1
  maxseq = 9

  
  
  numberOfCopies = np.random.randint(low=2, high=currentmaxnocopies, size=(batch_size, 1))


  input_data = np.zeros((batch_size, length*maxnumberofcopies, size), dtype=np.float32)
  target_output = np.zeros((batch_size, length*maxnumberofcopies, size), dtype=np.float32)

  sequence = genSeq((batch_size, length-1, size), min=minseq, max=maxseq, default=False)




  #print(batch_size, length, size, maxnumberofcopies, currentmaxnocopies)

  finputdata = np.zeros((batch_size, length*maxnumberofcopies, size))
  finputdata[:, :length-1, :] = sequence
  finputdata[:, length-1, :] = ENDSYM  # the end symbol
  finputdata[:, -2, :] = ENDSYM
  finputdata[:, -1, :] = numberOfCopies  # the end symbol

  for i in range(batch_size): # assure no empty sequences
    while np.sum(sequence[i]) == 0 or (testoccurance and st.isSaved(finputdata[i], flag="testData")):
      sequence[i] = genSeq((1, length-1, size), min=minseq, max=maxseq, default=False)
    
  input_data[:, :length-1, :] = sequence
  input_data[:, length-1, :] = ENDSYM  # the end symbol
  input_data[:, -2, :] = ENDSYM
  input_data[:, -1, :] = numberOfCopies  # the end symbol

  for i in range(batch_size): # save input data to avoid intersection between train, test and validation data
    for j in range(numberOfCopies[i][0]):
      target_output[i, (j*length)-j:((j+1)*length) - (j+1), :] = sequence[i]

  #debprint("inputdata:", input_data.shape)
  #debprint("outpudtata:",target_output.shape)


  return input_data, target_output 


def combLoss(output, target):
  return mse(output, target)

def incrementCurriculum(trainError, epoch, sequence_length, maxsequence_length, curriculum_fre):
  return epoch != 0 and sequence_length < maxsequence_length and epoch % curriculum_fre == 0



Setting up a new session...


In [36]:
batch_size = 100
sequence_max_length = 5
iterations = 1*10**4
summarize_freq = 100
check_freq = 500
curriculum_freq = 5000
curriculumMaxNoCopies_freq = 5000


# input_size = output_size = args.input_size
mem_slot = 16 # number of memory slots
mem_size = 1 # size of each memory slot
read_heads = 1


maxnumberofcopies=5
currentmaxnocopies=3

input_length = 6
input_size = 1#input_length*maxnumberofcopies #+10 memory = input
output_size = 64

In [37]:

#   dirname = os.path.dirname(__file__)
#   ckpts_dir = os.path.join(dirname, 'checkpoints')
#   if not os.path.isdir(ckpts_dir):
#     os.mkdir(ckpts_dir)

datas = []

name = 'mc_' + str(uuid4().hex)[:3] + ''

import os
def create_directory_if_not_exists(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        print("Directory created successfully!")
    else:
        print("Directory already exists.")

lastcp = None

create_directory_if_not_exists(name)

print(name)


debprint(input_size, output_size)

rnn = DNC(
        input_size=input_size,
        hidden_size=output_size,
        #rnn_type='lstm',
        rnn_type='rnn',
        num_layers=3,
        num_hidden_layers=1,
        dropout=0,
        nr_cells=mem_slot,
        cell_size=mem_size,
        read_heads=read_heads,
        gpu_id=-1,
        debug='store_true',
        batch_first=True,
        independent_linears=True
    )

with open(f'{name}/output.txt', 'w') as f:
  loadcp = False #= 'checkpoint_mc_631_66000.pth'
  if loadcp != False:
    rnn.load_state_dict(T.load(loadcp, weights_only=True))
    rnn.eval()

  print(rnn)

  last_save_losses = []



  optimizer = optim.Adam(rnn.parameters(), lr=0.001, eps=1e-9, betas=[0.9, 0.98]) # 0.0001

  print(sequence_max_length)

  for i in range(1, sequence_max_length+1):
    inputdataspace = 2**i*maxnumberofcopies # i bit numbers amd up to maxnumberofcopies
    testdatasize = int(inputdataspace*0.1)+1 # at least 1
    input_data, target_output = generate_data(testdatasize, i, maxnumberofcopies=maxnumberofcopies, currentmaxnocopies=maxnumberofcopies, testoccurance=False)

    for i in range(testdatasize):
      st.saveInput(input_data[i], output=target_output[i], withoutIncrement=True, flag="testData") 



  (chx, mhx, rv) = (None, None, None)
  Testloss = 0
  for epoch in tqdm(range(iterations + 1)):
    #llprint("\rIteration {ep}/{tot}".format(ep=epoch, tot=iterations))
    optimizer.zero_grad()

    random_length = np.random.randint(1, sequence_max_length + 1)
    input_data, target_output = generate_data(batch_size, random_length, maxnumberofcopies=maxnumberofcopies, currentmaxnocopies=currentmaxnocopies)
    input_data = var(T.from_numpy(input_data))
    target_output = var(T.from_numpy(target_output)) 
    # generate test data

    if rnn.debug:
      output, (chx, mhx, rv), v = rnn(input_data, (None, mhx, None), reset_experience=True, pass_through_memory=True)
    else:
      output, (chx, mhx, rv) = rnn(input_data, (None, mhx, None), reset_experience=True, pass_through_memory=True)


    loss = combLoss((output), target_output)

    if epoch % 100 == 0: # calculate test loss
      testset = st.getDataByFlag("testData") # get test data
      testlosses = []

      for k in range(int(len(testset) / batch_size)+1): # split to batches
        input_TEST_data = np.zeros((batch_size, (sequence_max_length+1)*maxnumberofcopies, 1), dtype=np.float32)
        target_TEST_output = np.zeros((batch_size, (sequence_max_length+1)*maxnumberofcopies, 1), dtype=np.float32)
        for i in range(batch_size):
          if i + k * batch_size < len(testset):
            sh1 = testset[k*batch_size+i]["input"].shape[0]
            sh2 = testset[k*batch_size+i]["output"].shape[0]
            input_TEST_data[i,:sh1] = testset[k*batch_size+i]["input"]
            target_TEST_output[i,:sh2] = testset[k*batch_size+i]["output"]
          else: # fill batch with random elements
            rel = random.choice(testset)
            sh1 = rel["input"].shape[0]
            sh2 = rel["output"].shape[0]
            input_TEST_data[i, :sh1] = rel["input"]
            target_TEST_output[i, :sh1] = rel["output"]
        input_TEST_data = var(T.from_numpy(input_TEST_data))
        target_TEST_output = var(T.from_numpy(target_TEST_output))

        TEST_output = np.zeros(target_TEST_output.shape)
        if rnn.debug:
          eTEST_output, _, _ = rnn(input_data, (None, mhx, None), reset_experience=True, pass_through_memory=True)
        else:
          eTEST_output, _ = rnn(input_data, (None, mhx, None), reset_experience=True, pass_through_memory=True)

        TEST_output[:eTEST_output.shape[0], :eTEST_output.shape[1], :eTEST_output.shape[2]] = eTEST_output.data.cpu().numpy()
        TEST_output = var(T.from_numpy(TEST_output))
        MyTestloss = combLoss((TEST_output), target_TEST_output).item() # calculate test loss
        testlosses.append(MyTestloss)
      Testloss = np.mean(testlosses) # calculate average

    datas.append({"epoch": epoch, "loss": loss.item(), "testloss": Testloss, "sequencelength": input_length})



    loss.backward()

    T.nn.utils.clip_grad_norm_(rnn.parameters(), 50)
    optimizer.step()
    loss_value = loss.item()

    summarize = (epoch % summarize_freq == 0)
    take_checkpoint = (epoch != 0) and (epoch % check_freq == 0)
    

    # detach memory from graph
    mhx = { k : (v.detach() if isinstance(v, var) else v) for k, v in mhx.items() }

    last_save_losses.append(loss_value)

    if summarize:
      loss = np.mean(last_save_losses)
      llprint("\n\tAvg. Loss: %.4f\n" % (loss))
      if np.isnan(loss):
        raise Exception('nan Loss')

    if summarize and rnn.debug:

      loss = np.mean(last_save_losses)
      last_save_losses = []

      viz.heatmap(
            v['memory'],
            opts=dict(
                xtickstep=10,
                ytickstep=2,
                title=name + 'Memory, t: ' + str(epoch) + ', loss: ' + str(loss),
                ylabel='layer * time',
                xlabel='mem_slot * mem_size'
            )
        )

      viz.heatmap(
            v['link_matrix'][-1].reshape(mem_slot, mem_slot),
            opts=dict(
                xtickstep=10,
                ytickstep=2,
                title=name + 'Link Matrix, t: ' + str(epoch) + ', loss: ' + str(loss),
                ylabel='mem_slot',
                xlabel='mem_slot'
            )
      )
     
      viz.heatmap(
            v['precedence'],
            opts=dict(
                xtickstep=10,
                ytickstep=2,
                title=name + 'Precedence, t: ' + str(epoch) + ', loss: ' + str(loss),
                ylabel='layer * time',
                xlabel='mem_slot'
            )
      )

    if incrementCurriculum(loss, epoch, currentmaxnocopies, maxnumberofcopies, curriculumMaxNoCopies_freq):
      currentmaxnocopies = currentmaxnocopies + 1
      if currentmaxnocopies > maxnumberofcopies:
        currentmaxnocopies = maxnumberofcopies
      print("Increasing max number of copies to " + str(currentmaxnocopies))


    if incrementCurriculum(loss, epoch, sequence_max_length, 5, curriculum_freq):
      sequence_max_length = sequence_max_length + 1
      print("Increasing max length to " + str(sequence_max_length))

    
    if summarize:
      random_length = np.random.randint(2, sequence_max_length + 1)
      input_data, target_output = generate_data(batch_size, random_length,  maxnumberofcopies=maxnumberofcopies, currentmaxnocopies=currentmaxnocopies)

      input_data = var(T.from_numpy(input_data))
      target_output = var(T.from_numpy(target_output))

      if rnn.debug:
        output, (chx, mhx, rv), v = rnn(input_data, (None, mhx, None), reset_experience=True, pass_through_memory=True)
      else:
        output, (chx, mhx, rv) = rnn(input_data, (None, mhx, None), reset_experience=True, pass_through_memory=True)

      print("\n\n")
      print("Input: ", torch.flatten(input_data[0]))
      print("Output: ", torch.flatten(torch.round(output[0], decimals=1)))
      print("Target: ", torch.flatten(target_output[0]))
      print("MSE Loss: ", str(mse(output, target_output).item()))
      print("CE Loss: ", str(criterion(output, target_output).item()))
      print("EXP Loss: ", str(exp_loss(output, target_output).item()))
      print("\n\n")
    
    if take_checkpoint:
      cur_weights = rnn.state_dict()
      T.save(cur_weights, f'{name}/checkpoint_{epoch}.pth')
      lastcp = f'{name}/checkpoint_{epoch}.pth'
      df = pd.DataFrame(datas)
      pickle.dump(df, open(f"{name}/df_{epoch}.pkl", "wb"))


  df = pd.DataFrame(datas) # plot loss 
  pickle.dump(df, open(f"{name}/df_total.pkl", "wb"))


  fig = go.Figure()
  fig.add_trace(go.Scatter(x=df["epoch"], y=df["loss"], mode='lines', name='Train Data'))
  fig.add_trace(go.Scatter(x=df["epoch"], y=df["testloss"], mode='lines', name='Test Data'))
  fig.update_layout(title='Losses', xaxis_title='Epoch', yaxis_title='Loss')
  fig.show()
  fig.write_html(f"{name}/losses.html")





Directory created successfully!
mc_915

----------------------------------------
DNC(1, 64, rnn_type=rnn, num_layers=3, num_hidden_layers=1, nr_cells=16, read_heads=1, cell_size=1, independent_linears=True, debug=store_true)
DNC(
  (rnn_layer_0): RNN(2, 64, batch_first=True)
  (rnn_layer_1): RNN(65, 64, batch_first=True)
  (rnn_layer_2): RNN(65, 64, batch_first=True)
  (rnn_layer_memory_shared): Memory(
    (read_keys_transform): Linear(in_features=64, out_features=1, bias=True)
    (read_strengths_transform): Linear(in_features=64, out_features=1, bias=True)
    (write_key_transform): Linear(in_features=64, out_features=1, bias=True)
    (write_strength_transform): Linear(in_features=64, out_features=1, bias=True)
    (erase_vector_transform): Linear(in_features=64, out_features=1, bias=True)
    (write_vector_transform): Linear(in_features=64, out_features=1, bias=True)
    (free_gates_transform): Linear(in_features=64, out_features=1, bias=True)
    (allocation_gate_transform): Line

  0%|          | 0/10001 [00:00<?, ?it/s]


	Avg. Loss: 7.2999


  0%|          | 1/10001 [00:01<4:28:53,  1.61s/it]




Input:  tensor([ 8.,  6., -9.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -9.,
         2.])
Output:  tensor([0.3000, 0.4000, 0.4000, 0.3000, 0.3000, 0.4000, 0.3000, 0.3000, 0.4000,
        0.4000, 0.4000, 0.4000, 0.4000, 0.3000, 0.2000],
       grad_fn=<ViewBackward0>)
Target:  tensor([8., 6., 8., 6., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
MSE Loss:  6.452544689178467
CE Loss:  0.4688805639743805
EXP Loss:  117.891845703125





...




Input:  tensor([ 8.,  4.,  3.,  3., -9.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -9.,  2.])
Output:  tensor([ 8.0000,  4.2000,  3.1000,  2.6000,  7.7000,  3.9000,  2.6000,  2.8000,
         3.9000,  1.4000,  1.5000,  0.7000, -0.1000, -0.0000, -0.1000, -0.1000,
        -0.1000, -0.1000, -0.1000, -0.1000, -0.1000, -0.1000, -0.1000, -0.1000,
        -0.1000], grad_fn=<ViewBackward0>)
Target:  tensor([8., 4., 3., 3., 8., 4., 3., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.])
MSE Loss:  1.0808510780334473
CE Loss:  -6.666878700256348
EXP Loss:  3.0408401489257812





100%|█████████▉| 10000/10001 [1:41:01<00:00,  1.61it/s]


	Avg. Loss: 0.9237
Increasing max number of copies to 5


100%|██████████| 10001/10001 [1:41:02<00:00,  1.65it/s]




Input:  tensor([ 5.,  6., -9.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -9.,
         3.])
Output:  tensor([4.9000, 6.0000, 4.7000, 5.9000, 2.3000, 2.7000, -0.0000, 0.0000, -0.0000,
        0.0000, 0.0000, 0.0000, -0.0000, -0.0000, 0.0000],
       grad_fn=<ViewBackward0>)
Target:  tensor([5., 6., 5., 6., 5., 6., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
MSE Loss:  1.9385112524032593
CE Loss:  -5.405060768127441
EXP Loss:  27.467405319213867





In [38]:
#from dnc.lib import calcAccuracy
print(name)

if 'rnn' in locals() or 'rnn' in globals():
  del rnn

rnn = DNC(
        input_size=input_size,
        hidden_size=output_size,
        rnn_type='rnn',
        #rnn_type='lstm',
        num_layers=3,
        num_hidden_layers=1,
        dropout=0,
        nr_cells=mem_slot,
        cell_size=mem_size,
        read_heads=read_heads,
        gpu_id=-1,
        debug='store_true',
        batch_first=True,
        independent_linears=True,
        nonlinearity='tanh',
    )

#name = 'mc_7ac'
lastcp = f'{name}/checkpoint_{iterations}.pth'

with open(f"{name}/output_2.txt", "w") as f:
  batch_size=1
  rnn.load_state_dict(T.load(lastcp, weights_only=True))
  rnn.eval()

  maxnumberofcopies = 3
  input_data, target_output = generate_data(1, np.random.randint(2, 4), maxnumberofcopies=maxnumberofcopies, currentmaxnocopies=maxnumberofcopies)

  input_data = var(T.from_numpy(input_data))
  target_output = var(T.from_numpy(target_output))
  print(input_data)
  print(target_output)

  stepByStep = copy.deepcopy(STEPBYSTEPOBJ)
  stepByStep["CurrI"] = 0
  stepByStep["input"] = input_data.detach().numpy()
  stepByStep["target"] = target_output.detach().numpy()
  stepByStep["currentObj"] = copy.deepcopy(stepByStep["defObj"])
  stepByStep["currentObj"]["i"] = 0 
  stepByStep["MEMORYCOLUMNS"] = mem_slot
  stepByStep["INPUTSIZE"] = input_size
  stepByStep["OUTPUTSIZE"] = output_size

  output, (chx, mhx, rv), v = rnn(input_data, (None, None, None), reset_experience=True, pass_through_memory=True, stepByStep=stepByStep)

  stepByStep['output'] = output.detach().numpy()
  stepByStep["objects"].append(copy.deepcopy(stepByStep["currentObj"]))
  stepByStep['loss'] = str(mse(output, target_output).item())

  pickle.dump(stepByStep, open(f"{name}/stepByStep.pkl", "wb"))



  for i in range(1):#range(int((iterations + 1) / 100)):
    llprint("\nIteration %d/%d" % (i, iterations))
    # We test now the learned generalization using sequence_max_length examples
    random_length = np.random.randint(2, sequence_max_length + 1)
    input_data, target_output = generate_data(batch_size, random_length, maxnumberofcopies=maxnumberofcopies, currentmaxnocopies=maxnumberofcopies)


    input_data = var(T.from_numpy(input_data))
    target_output = var(T.from_numpy(target_output))
    if rnn.debug:
      output, (chx, mhx, rv), v = rnn(input_data, (None, mhx, None), reset_experience=True, pass_through_memory=True)
    else:
      output, (chx, mhx, rv) = rnn(input_data, (None, mhx, None), reset_experience=True, pass_through_memory=True)

    print("\n\n")
    print("Input: ", torch.flatten(input_data[0]))
    print("Output: ", torch.flatten(torch.round(output[0], decimals=1)))
    print("Target: ", torch.flatten(target_output[0]))
    print("MSE Loss: ", str(mse(output, target_output).item()))
    print("CE Loss: ", str(criterion(output, target_output).item()))
    print("EXP Loss: ", str(exp_loss(output, target_output).item()))
    #print("accuracy: ", str(calcAccuracy(output, target_output).item()))
    print("\n\n")
    output = output[:, -1, :].sum().data.cpu().numpy()
    target_output = target_output.sum().data.cpu().numpy()

    
    try:
      print("\nReal value: ", ' = ' + str(int(target_output[0])))
      print("Predicted:  ", ' = ' + str(int(output // 1)) + " [" + str(output) + "]")
    except Exception as e:
      pass

mc_915
tensor([[[ 5.],
         [ 5.],
         [-9.],
         [ 0.],
         [ 0.],
         [ 0.],
         [ 0.],
         [-9.],
         [ 2.]]])
tensor([[[5.],
         [5.],
         [5.],
         [5.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]]])

Iteration 0/10000


Input:  tensor([ 5.,  2.,  6.,  4., -9.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -9.,
         2.])
Output:  tensor([5.7000, 2.2000, 5.8000, 4.2000, 5.3000, 2.5000, 5.6000, 3.8000, 2.8000,
        0.8000, 2.9000, 1.9000, -0.0000, -0.0000, 0.0000],
       grad_fn=<ViewBackward0>)
Target:  tensor([5., 2., 6., 4., 5., 2., 6., 4., 0., 0., 0., 0., 0., 0., 0.])
MSE Loss:  1.4620453119277954
CE Loss:  -7.879790306091309
EXP Loss:  2.9134232997894287



